# Assignment 3: Transformer is All You Need

Federico Giorgi (fg2617)

## Basics

In [1]:
# Import all the libraries
import os, math, torch, urllib.request
from dataclasses import dataclass

from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.cuda.amp import autocast, GradScaler

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

device = "cuda" if torch.cuda.is_available() else "cpu"

## 1 Data Preparation

### 1.1 Load the Tiny Shakespeare text

In [2]:
# Define local path for dataset
TINY_PATH = "tiny_shakespeare.txt"

# Download the Tiny Shakespeare corpus from Karpathy’s repo
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt",
    TINY_PATH
)

# Read the corpus into memory as a single string
with open(TINY_PATH, "r", encoding="utf-8") as f:
    corpus_text = f.read()

# Print the first 100 characters
print(corpus_text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


### 1.2 Tokenization

In [3]:
# Define special tokens for padding, unknown words, and sequence boundaries
special_tokens = ["[PAD]", "[UNK]", "[BOS]", "[EOS]"]

# Initialize a Byte Pair Encoding (BPE) tokenizer with an unknown token
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

# Split text initially on whitespace before BPE merges are applied
tokenizer.pre_tokenizer = Whitespace()

# Train the BPE tokenizer on the corpus, limiting vocabulary size to 500 tokens
trainer = BpeTrainer(vocab_size=500, min_frequency=2, special_tokens=special_tokens)
tokenizer.train_from_iterator([corpus_text], trainer=trainer)

# Retrieve token IDs for the special tokens and vocabulary size
pad_id  = tokenizer.token_to_id("[PAD]")
bos_id  = tokenizer.token_to_id("[BOS]")
eos_id  = tokenizer.token_to_id("[EOS]")
vocab_size = tokenizer.get_vocab_size()

# Encode entire corpus to integer IDs
ids = tokenizer.encode(corpus_text).ids

### 1.3 Sequence formatting

In [4]:
# For next-token prediction: Input = first N tokens, Target = same sequence shifted by 1

def make_windows(token_ids, seq_len):
    # Total number of tokens in the corpus
    L = len(token_ids)

    # We need i + seq_len + 1 <= L so that both input and target fit
    limit = L - (seq_len + 1)

    # Lists to collect all input and target sequences
    inputs = []
    targets = []

    # Slide a window of size (seq_len + 1) across the corpus with stride 1
    for i in range(0, max(0, limit + 1)):
        # Take a chunk of length seq_len + 1
        seq = token_ids[i : i + seq_len + 1]

        # Input: all tokens except the last one
        inp = seq[:-1]

        # Target: all tokens except the first one (shifted by one position)
        tgt = seq[1:]

        # Append the pair to the dataset lists
        inputs.append(inp)
        targets.append(tgt)

    return inputs, targets

# Generate all (input, target) pairs from the encoded corpus
SEQ_LEN = 50
inputs, targets = make_windows(ids, SEQ_LEN)

In [5]:
# Quick sanity check for the windowed dataset
print(f"Total number of input-target pairs: {len(inputs)}")

# Print the first pair to verify correct shifting
print("\nExample pair:")
print("Input IDs :", inputs[0])
print("Target IDs:", targets[0])

# Decode the first example back to text (optional, for interpretability)
print("\nDecoded example:")
print("Input text :", tokenizer.decode(inputs[0]))
print("Target text:", tokenizer.decode(targets[0]))


Total number of input-target pairs: 448079

Example pair:
Input IDs : [407, 17, 93, 49, 66, 75, 12, 396, 283, 116, 300, 101, 110, 424, 46, 138, 131, 8, 395, 81, 365, 10, 15, 80, 12, 33, 154, 252, 8, 365, 10, 407, 17, 93, 49, 66, 75, 12, 318, 158, 135, 58, 86, 316, 92, 44, 145, 131, 82, 277]
Target IDs: [17, 93, 49, 66, 75, 12, 396, 283, 116, 300, 101, 110, 424, 46, 138, 131, 8, 395, 81, 365, 10, 15, 80, 12, 33, 154, 252, 8, 365, 10, 407, 17, 93, 49, 66, 75, 12, 318, 158, 135, 58, 86, 316, 92, 44, 145, 131, 82, 277, 45]

Decoded example:
Input text : First C it i z en : Be fore we pro ce ed any f ur ther , hear me speak . A ll : S pe ak , speak . First C it i z en : You are all r es ol ve d ra ther to di
Target text: C it i z en : Be fore we pro ce ed any f ur ther , hear me speak . A ll : S pe ak , speak . First C it i z en : You are all r es ol ve d ra ther to di e


### 1. 4 Data split

In [6]:
# Total number of (input, target) pairs produced by windowing
dataset_size = len(inputs)

# 80/20 split
train_size = int(0.8 * dataset_size)
val_size   = dataset_size - train_size

In [7]:
class NextTokenDataset(Dataset):
    """
    Simple Dataset wrapper for next-token prediction.
    Stores precomputed windows as LongTensors.
    """
    def __init__(self, X, Y):
        # Cast each window to torch.LongTensor (needed by nn.Embedding and CE loss)
        self.X = [torch.tensor(x, dtype=torch.long) for x in X]
        self.Y = [torch.tensor(y, dtype=torch.long) for y in Y]

    def __len__(self):
        # Number of (input, target) pairs
        return len(self.X)

    def __getitem__(self, i):
        # Return one pair shaped (T,), (T,) where T=SEQ_LEN
        return self.X[i], self.Y[i]

# Full dataset (deterministic split via fixed seed)
full_ds = NextTokenDataset(inputs, targets)
train_ds, val_ds = random_split(
    full_ds,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

def collate_batch(batch):
    """
    Collate function for DataLoader.
    Sequences are already fixed-length (SEQ_LEN), so we can stack them directly.

    Input:
        batch: list of N items, each is (x, y) with shape (T,), (T,)
    Output:
        X: (B, T), Y: (B, T)
    """
    # b[0] is input tensor, b[1] is target tensor
    X = torch.stack([b[0] for b in batch], dim=0)  # (B, T)
    Y = torch.stack([b[1] for b in batch], dim=0)  # (B, T)
    return X, Y

BATCH_SIZE = 128

# DataLoaders for training and validation.
# Note: drop_last=True on train for consistent batch size (helps with certain training loops).
train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_batch
)

# For validation, keep all examples (no shuffling, no drop_last)
val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    drop_last=False,
    collate_fn=collate_batch
)

In [8]:
# Quick sanity checks
xb, yb = next(iter(train_loader))
print(f"Train batches: {len(train_loader)}  |  Val batches: {len(val_loader)}")
print(f"Batch shapes  -> X: {xb.shape}, Y: {yb.shape}  (expect: ({BATCH_SIZE}, {SEQ_LEN}))")
assert xb.shape == yb.shape == (BATCH_SIZE, SEQ_LEN)

Train batches: 2800  |  Val batches: 701
Batch shapes  -> X: torch.Size([128, 50]), Y: torch.Size([128, 50])  (expect: (128, 50))


### 1.5 Token embedding

In [9]:
### 1.5 Token embedding (sinusoidal positions only)

# We map token IDs -> dense vectors with nn.Embedding,
# then add fixed (non-trainable) sinusoidal positional encodings à la "Attention Is All You Need".

class TokenEmbedding(nn.Module):
    """
    Token embedding table: converts token IDs to d_model-dimensional vectors.
    """
    def __init__(self, vocab_size: int, d_model: int):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, T) of dtype long
        return self.embed(x)  # (B, T, d_model)


class SinusoidalPositionalEncoding(nn.Module):
    """
    Fixed sinusoidal positional encodings (no learned parameters).
    Follows Vaswani et al. (2017). Register as a buffer so it moves with .to(device)
    but is excluded from optimizer updates.
    """
    def __init__(self, d_model: int, max_len: int = 10000):
        super().__init__()
        # Create table of shape (max_len, d_model)
        pe = torch.zeros(max_len, d_model, dtype=torch.float32)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)        # (max_len, 1)
        # Frequencies for even/odd dimensions
        div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float32) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # even dims
        pe[:, 1::2] = torch.cos(position * div_term)  # odd dims

        # Register as buffer (not a Parameter): no gradients, moves with the module across devices
        self.register_buffer("pe", pe)  # shape: (max_len, d_model)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Add positional encodings for the first T positions.
        x is expected to be (B, T, d_model).
        """
        B, T, D = x.shape
        # Slice the first T rows, then broadcast over batch dimension
        return x + self.pe[:T].unsqueeze(0)  # (1, T, D) + (B, T, D) -> (B, T, D)


# ----- build embeddings and run a quick test on one batch -----
device   = "cuda" if torch.cuda.is_available() else "cpu"
d_model  = 256
max_len  = max(SEQ_LEN, 512)  # keep some headroom if you later increase context length

tok_emb  = TokenEmbedding(vocab_size, d_model).to(device)
pos_enc  = SinusoidalPositionalEncoding(d_model, max_len=max_len).to(device)

# Fetch one batch of IDs (B, T)
xb, yb = next(iter(train_loader))
xb = xb.to(device)  # (B, T)

# Token embeddings: (B, T, d_model)
h_tok = tok_emb(xb)

# Add sinusoidal positions: (B, T, d_model)
h = pos_enc(h_tok)

In [10]:
print("Sanity checks for embeddings + sinusoidal positions:")
print(" - xb dtype / shape:", xb.dtype, xb.shape)
print(" - token only     :", h_tok.shape)
print(" - token+position :", h.shape)

# 1) Shapes must match
assert h_tok.shape == (xb.size(0), xb.size(1), d_model)
assert h.shape     == (xb.size(0), xb.size(1), d_model)

# 2) Positional buffer should not require gradients
assert pos_enc.pe.requires_grad is False

# 3) Same position across different sequences has identical positional vector
#    (i.e., h[b, t, :] - h_tok[b, t, :] equals the same vector for all b, fixed t)
with torch.no_grad():
    t = 0  # check the first position; you can try other t as well
    pos_vecs = h[:, t, :] - h_tok[:, t, :]              # (B, D)
    diff = (pos_vecs - pos_vecs[0]).abs().max().item()  # max deviation from the first sample
    print(f" - max deviation of pos[t={t}] across batch: {diff:.3e}")
    assert diff < 1e-6

# 4) Different positions within the same sequence must have different positional vectors
with torch.no_grad():
    if xb.size(1) >= 2:
        # Compare pos enc at t=0 vs t=1 for the first sample in batch
        v0 = h[0, 0, :] - h_tok[0, 0, :]
        v1 = h[0, 1, :] - h_tok[0, 1, :]
        cosine = torch.nn.functional.cosine_similarity(v0, v1, dim=0).item()
        print(f" - cosine similarity between pos[0] and pos[1]: {cosine:.4f} (should be < 1.0)")

Sanity checks for embeddings + sinusoidal positions:
 - xb dtype / shape: torch.int64 torch.Size([128, 50])
 - token only     : torch.Size([128, 50, 256])
 - token+position : torch.Size([128, 50, 256])
 - max deviation of pos[t=0] across batch: 2.384e-07
 - cosine similarity between pos[0] and pos[1]: 0.9721 (should be < 1.0)


## 2 Tiny Transformer Implementation

In [11]:
class SelfAttention(nn.Module):
    """Self-attention with causal masking"""
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        self.W_q = nn.Linear(d_model, d_model, bias=False)
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)

    def forward(self, x):
        batch_size, seq_len, d_model = x.shape

        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)

        scores = Q @ K.transpose(-2, -1) / (d_model ** 0.5)

        # Causal mask
        mask = torch.tril(torch.ones(seq_len, seq_len)).to(x.device)
        scores = scores.masked_fill(mask == 0, float('-inf'))

        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = torch.nan_to_num(attn_weights, 0.0)

        output = attn_weights @ V
        return output, attn_weights


class FeedForward(nn.Module):
    """Two-layer MLP"""
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))


class TransformerBlock(nn.Module):
    """One transformer block: Attention + FFN + Residuals"""
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.attention = SelfAttention(d_model)
        self.ffn = FeedForward(d_model, d_ff)

    def forward(self, x):
        attn_out, attn_weights = self.attention(x)
        x = x + attn_out  # Residual

        ffn_out = self.ffn(x)
        x = x + ffn_out  # Residual

        return x, attn_weights


class TinyTransformer(nn.Module):
    """Complete transformer model"""
    def __init__(self, vocab_size, d_model, n_layers, d_ff, max_seq_len=128):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = nn.Parameter(torch.randn(1, max_seq_len, d_model) * 0.02)

        self.blocks = nn.ModuleList([
            TransformerBlock(d_model, d_ff) for _ in range(n_layers)
        ])

        self.output_layer = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        seq_len = x.shape[1]
        x = self.token_embedding(x) + self.pos_encoding[:, :seq_len, :]

        all_attn = []
        for block in self.blocks:
            x, attn = block(x)
            all_attn.append(attn)

        logits = self.output_layer(x)
        return logits, all_attn

## 3 Visualization & Interpretation

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print(f"✓ Using GPU: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠ No GPU found, using CPU")

d_model = 128
n_layers = 2
d_ff = 512
learning_rate = 0.001
n_epochs = 1000
batch_size = 4096

model = TinyTransformer(vocab_size, d_model, n_layers, d_ff, max_len).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=50)

X_padded = X_padded.to(device)
Y_tensor = Y_tensor.to(device)

scaler = GradScaler()

print("\n" + "="*70)
print("MODEL ARCHITECTURE")
print("="*70)
print(f"Device: {device}")
print(f"Embedding dim: {d_model}")
print(f"Layers: {n_layers}")
print(f"FFN hidden: {d_ff}")
print(f"Batch size: {batch_size}")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Epochs: {n_epochs}")

print("\n" + "="*70)
print("TRAINING")
print("="*70)

train_losses = []
train_accs = []
best_loss = float('inf')
n_samples = len(X_padded)

for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    n_batches = 0

    indices = torch.randperm(n_samples, device=device)

    for i in range(0, n_samples, batch_size):
        batch_idx = indices[i:min(i+batch_size, n_samples)]
        batch_X = X_padded[batch_idx]
        batch_Y = Y_tensor[batch_idx]

        optimizer.zero_grad()

        with autocast():
            logits, _ = model(batch_X)
            predictions = logits[:, -1, :]
            loss = criterion(predictions, batch_Y)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()

        pred_tokens = torch.argmax(predictions, dim=1)
        acc = (pred_tokens == batch_Y).float().mean().item()

        epoch_loss += loss.item()
        epoch_acc += acc
        n_batches += 1

    avg_loss = epoch_loss / n_batches
    avg_acc = epoch_acc / n_batches

    train_losses.append(avg_loss)
    train_accs.append(avg_acc)

    old_lr = optimizer.param_groups[0]['lr']
    scheduler.step(avg_loss)
    new_lr = optimizer.param_groups[0]['lr']

    if new_lr != old_lr:
        print(f"    → LR reduced: {old_lr:.6f} → {new_lr:.6f}")

    if avg_loss < best_loss:
        best_loss = avg_loss
        best_epoch = epoch + 1

    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch+1:4d}/{n_epochs} | Loss: {avg_loss:.4f} | Acc: {avg_acc:.2%}")
        if device.type == 'cuda':
            print(f"    GPU Memory: {torch.cuda.memory_allocated()/1e9:.2f}GB / {torch.cuda.max_memory_allocated()/1e9:.2f}GB")

    if avg_loss < 0.01:
        print(f"\n✓ Excellent! Stopping at epoch {epoch+1}")
        break

print(f"\n✓ Training complete!")
print(f"Best: Loss={best_loss:.4f} at epoch {best_epoch}")
print(f"Final: Loss={train_losses[-1]:.4f}, Acc={train_accs[-1]:.2%}")

# 清理GPU缓存
if device.type == 'cuda':
    torch.cuda.empty_cache()


# ============================================================
# 5. Visualization: Training Progress
# ============================================================

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(train_losses, linewidth=2, color='steelblue')
ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax1.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax1.set_title('Training Loss', fontsize=14, fontweight='bold')
ax1.grid(alpha=0.3)

ax2.plot(train_accs, linewidth=2, color='seagreen')
ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax2.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax2.set_title('Training Accuracy', fontsize=14, fontweight='bold')
ax2.set_ylim([0, 1])
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================================
# 6. Inference
# ============================================================

import ipywidgets as widgets
from IPython.display import display, clear_output
import torch.nn.functional as F

def predict_next(prompt, top_k=5):
    model.eval()
    with torch.no_grad():
        tokens = [word_to_id[w] for w in prompt.lower().split() if w in word_to_id]
        if not tokens:
            return []

        input_tensor = torch.tensor([tokens])
        logits, _ = model(input_tensor)
        probs = F.softmax(logits[0, -1, :], dim=0)

        top_probs, top_idx = torch.topk(probs, min(top_k, vocab_size))
        return [(id_to_word[i.item()], p.item()) for i, p in zip(top_idx, top_probs)]

# Test with predefined prompts
test_prompts = [
    "the sun rises",
    "birds fly south",
    "water boils at",
    "plants need sunlight"
]

print("\n" + "="*70)
print("INFERENCE: NEXT TOKEN PREDICTIONS")
print("="*70)

for prompt in test_prompts:
    preds = predict_next(prompt)
    print(f"\nPrompt: '{prompt}'")
    for i, (word, prob) in enumerate(preds, 1):
        bar = '█' * int(prob * 40)
        print(f"  {i}. {word:15s} {prob:.3f} {bar}")

# Create interactive widget
print("\n" + "="*70)
print("INTERACTIVE NEXT TOKEN PREDICTOR")
print("="*70)

# Create input text box
text_input = widgets.Text(
    value='',
    placeholder='Enter text here...',
    description='Input:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Create slider for top-k
topk_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=10,
    step=1,
    description='Top-K:',
    style={'description_width': 'initial'}
)

# Create output area
output_area = widgets.Output()

# Create button for prediction
predict_button = widgets.Button(
    description='Predict Next Token',
    button_style='primary',
    tooltip='Click to predict next token'
)

def on_predict(b=None):
    with output_area:
        clear_output(wait=True)
        prompt = text_input.value.strip()

        if not prompt:
            print("⚠️ Please enter some text!")
            return

        top_k = topk_slider.value
        preds = predict_next(prompt, top_k)

        if not preds:
            print("❌ No valid words found in vocabulary!")
            return

        # Display results with nice formatting
        print(f"📝 Input: '{prompt}'\n")
        print("🎯 Next Token Predictions:\n")

        # Find max probability for scaling bars
        max_prob = preds[0][1] if preds else 1

        for i, (word, prob) in enumerate(preds, 1):
            # Create probability bar
            bar_length = int(prob / max_prob * 30)
            bar = '█' * bar_length + '░' * (30 - bar_length)

            # Color coding based on probability
            if prob > 0.5:
                emoji = "🟢"
            elif prob > 0.2:
                emoji = "🟡"
            else:
                emoji = "🔴"

            print(f"  {emoji} {i:2d}. {word:15s} [{bar}] {prob:.3%}")

        print(f"\n💡 Tip: The model predicts '{preds[0][0]}' as the most likely next token")

# Bind button click event
predict_button.on_click(on_predict)

# Also trigger prediction when pressing Enter in text box
def on_text_change(change):
    if change['name'] == 'value' and change['new'] and change['new'][-1:] == '\n':
        text_input.value = text_input.value.strip()
        on_predict()

text_input.observe(on_text_change)

# Create layout
title = widgets.HTML(value="<h3>🤖 Interactive Next Token Predictor</h3>")
instructions = widgets.HTML(value="<p><i>Type a sentence fragment and see what the model predicts as the next token!</i></p>")

# Arrange widgets
input_box = widgets.HBox([text_input, predict_button])
controls = widgets.VBox([topk_slider])

# Display everything
display(title)
display(instructions)
display(input_box)
display(controls)
display(output_area)

# Show initial example
text_input.value = "the sun"
on_predict()